**Michael Vizelman**

## Analyzing Chess Tournament Results ##

**The Task:**  
Given the text file (“tournamentinfo.txt”) with chess tournament results.   
Create a Jupyter Notebook that generates a .CSV file with the following information for all of the chess players:  

| Player’s Name | Player’s State | Total Number of Points | Player’s Pre-Rating | Average Pre Tournament Chess Rating of Opponents |
| --- | --- | --- | --- | --- |
| Gary Hua | ON | 6.0 | 1794 | 1605 |
| chess player 2 | ... | ... | ... | ... |
| chess player 3 | ... | ... | ... | ... |
| ... | ... | ... | ... | ... |

In [1]:
# load the pandas + numpy libraries
import pandas as pd
import numpy as np
# load the re module
import re

**The first steps are:**
- Upload the text file (“tournamentinfo.txt”) to GitHub.
- Upload the file from GitHub into a dataframe, using read_csv, without a header.
- Then:
    - create regex pattern `'\--+'` that will match the hyphen symbols `'---'` seperating between the players.
    - create a boolean list, that will allow us to choose which rows to drop.
    - subset the datatframe without the unwanted seperation rows, using the boolean list.
    - drop first two header rows.

In [2]:
f=pd.read_csv('https://raw.githubusercontent.com/vizelman/Projects/main/Data_Files/tournamentinfo.txt?token=ANAIKVKRWL7V3SUYHIIBYNTABROMG',header=None)

#save the pattern we want to delete in a list
patternDel = "\--+"

#select all the rows, and create a boolean list where "True" stands for a row that contains the pattern
#and "False" stands for a row that doesn't contain the pattern
f_wr = f[0].str.contains(patternDel)

#now, select only the rows that don't contain the pattern - only "False" rows using the boolean list,
#from the original dataframe
f_new=f[~f_wr]

#now we drop the first two rows, because they are headers and don't contain data.
f_new=f_new.drop([1,2])

#print the new dataframe to check what we've achieved so far
f_new.head()

,0
4,1 | GARY HUA |6.0 ...
5,ON | 15445895 / R: 1794 ->1817 |N:2 ...
7,2 | DAKSHESH DARURI |6.0 ...
8,MI | 14598900 / R: 1553 ->1663 |N:2 ...
10,3 | ADITYA BAJAJ |6.0 ...


**The next steps are:**
- create regex pattern `'|'` that will match the symbol `'|'`, seperating between the data points in the column.
- use the `split()` method and ` expand` method to return the splitted string into separate columns.

In [3]:
#now we split the column containing all the data into seperate coloumns, 
#using regex '|' as the separator between the data points 
#we save the result into a new dataframe
new = f_new[0].str.split("|", expand = True) 

#print the new dataframe to check what we've achieved so far
new.head()

,0,1,2,3,4,5,6,7,8,9,10
4,1,GARY HUA,6.0,W 39,W 21,W 18,W 14,W 7,D 12,D 4,
5,ON,15445895 / R: 1794 ->1817,N:2,W,B,W,B,W,B,W,
7,2,DAKSHESH DARURI,6.0,W 63,W 58,L 4,W 17,W 16,W 20,W 7,
8,MI,14598900 / R: 1553 ->1663,N:2,B,W,B,W,B,W,B,
10,3,ADITYA BAJAJ,6.0,L 8,W 61,W 25,W 21,W 11,W 13,W 12,


**The next steps are:**

- create regex pattern `'^\s+[0-9]'` that will match the players' IDs, to identify tournament results rows and player information rows.   
- create a new dataframe containing only the player information rows.
    - reset the index on the dataframe, to work with it as a key
    - drop all the columns containing the data we don't need
    - create regex pattern `'/|->|:'` that will match the seperating symbols `'/' or '->' or ':'`, seperating between the data points in the column with the data we need and other data. 
    - use the `split()` method and expand method to return the splitted string into separate columns.
    - drop all the columns containing the data we don't need.
    - give the column containing the players' State a meaningful name 'State'.
    - use regex `'([0-9]+)'` to extract the 'Pre_rating' number and set it as an `int32` number.

In [4]:
#now, we want to identify the rows with the results of each tournament
#and identify the rows with the player information.
#since the rows that have the player ID are the tournament results rows,
#we decided to use regex that will match white space and a number to identify them.
row_player=re.compile("^\s+[0-9]")

#as we privously did, we save the result of the boolean test in a list,
#if the rows contain tournament results than"True" else "False"
player=new[0].str.contains(row_player)

#print boolean list to check it contains "True" and "False" alternately, as expected 
player.head()

4      True
5     False
7      True
8     False
10     True
Name: 0, dtype: bool

- create a new dataframe containing only the player information rows.

In [5]:
#now we create a new dataframe with only the rows of the player information,
#using the player list we created choose only the "False" rows (always the second row for each player in our data)
second_row=new[~player]

#print dataframe to check it contains the expected rows 
second_row.head()

,0,1,2,3,4,5,6,7,8,9,10
5,ON,15445895 / R: 1794 ->1817,N:2,W,B,W,B,W,B,W,
8,MI,14598900 / R: 1553 ->1663,N:2,B,W,B,W,B,W,B,
11,MI,14959604 / R: 1384 ->1640,N:2,W,B,W,B,W,B,W,
14,MI,12616049 / R: 1716 ->1744,N:2,W,B,W,B,W,B,B,
17,MI,14601533 / R: 1655 ->1690,N:2,B,W,B,W,B,W,B,


- reset the index on the dataframe, to work with it as a key.

In [6]:
#we reset the index, to work with it as a key later
second_row=second_row.reset_index(drop=True)

#print dataframe to check result is as expected
second_row.head()

,0,1,2,3,4,5,6,7,8,9,10
0,ON,15445895 / R: 1794 ->1817,N:2,W,B,W,B,W,B,W,
1,MI,14598900 / R: 1553 ->1663,N:2,B,W,B,W,B,W,B,
2,MI,14959604 / R: 1384 ->1640,N:2,W,B,W,B,W,B,W,
3,MI,12616049 / R: 1716 ->1744,N:2,W,B,W,B,W,B,B,
4,MI,14601533 / R: 1655 ->1690,N:2,B,W,B,W,B,W,B,


- drop all the columns containing the data we don't need

In [7]:
#now we drop all the columns that are not in our scope
second_row=second_row.drop(columns=[2,3,4,5,6,7,8,9,10],axis=1)

#print dataframe to check result is as expected
second_row.head()

,0,1
0,ON,15445895 / R: 1794 ->1817
1,MI,14598900 / R: 1553 ->1663
2,MI,14959604 / R: 1384 ->1640
3,MI,12616049 / R: 1716 ->1744
4,MI,14601533 / R: 1655 ->1690


- create regex pattern `'/|->|:'` that will match the seperating symbols `'/' or '->' or ':'`, seperating between the data points in the column with the data we need and other data. 
- use the `'split()'` method and `'expand'` method to return the splitted string into separate columns.
- drop all the columns containing the data we don't need.
- give the column containing the players State a meaningful name 'State'.
- use regex `'([0-9]+)'` to extract the 'Pre_rating' number and set it as an `int32` number.

In [8]:
#now we split column 1, to obtain the value we want 'Pre_rating' from all the values which are:
#'USCF_ID','R','Pre_rating','Post_rating'
#to do that, we use the split the string into the variables 
# using a regex that will match all the possible seperating characters.
second_row[['USCF_ID','R','Pre_rating','Post_rating']]=second_row[1].str.split("/|->|:", expand = True) 

#we drop the columns not in our scope
second_row=second_row.drop(columns=[1,'R','Post_rating','USCF_ID'],axis=1)

#rename column 0, as 'State'
second_row.rename(columns={0:'State',
                    },inplace = True)

#we need the 'Pre_rating' to be a number, and to extract only the number
#we achieve that using regex '([0-9]+)'
second_row['Pre_rating']=second_row['Pre_rating'].str.extract('([0-9]+)').astype('int32')

#print dataframe to check result is as expected
second_row.head()

,State,Pre_rating
0,ON,1794
1,MI,1553
2,MI,1384
3,MI,1716
4,MI,1655


**The next steps are:**
- create a new dataframe containing only the tournament results rows.
    - reset the index on the dataframe, to work with it as a key.
    - split the result-letter-indicator from the ID of the opponent, using the `'extract()'` method with regex `'([0-9]+)'` to return only the ID.
    - drop all the columns containing the data we don't need.
    - give the columns containing the data we need meaningful names.

In [9]:
#now we create a new dataframe with only the rows of the tournament results,
#using the player list we created choose only the "True" rows (always the first row for each player in our data)
first_row=new[player]

#print dataframe to check result is as expected
first_row.head()

,0,1,2,3,4,5,6,7,8,9,10
4,1,GARY HUA,6.0,W 39,W 21,W 18,W 14,W 7,D 12,D 4,
7,2,DAKSHESH DARURI,6.0,W 63,W 58,L 4,W 17,W 16,W 20,W 7,
10,3,ADITYA BAJAJ,6.0,L 8,W 61,W 25,W 21,W 11,W 13,W 12,
13,4,PATRICK H SCHILLING,5.5,W 23,D 28,W 2,W 26,D 5,W 19,D 1,
16,5,HANSHI ZUO,5.5,W 45,W 37,D 12,D 13,D 4,W 14,W 17,


- reset the index on the dataframe, to work with it as a key.

In [10]:
#we reset the index on the dataframe, to work with it as a key
first_row=first_row.reset_index(drop=True)

#print dataframe to check result is as expected
first_row.head()

,0,1,2,3,4,5,6,7,8,9,10
0,1,GARY HUA,6.0,W 39,W 21,W 18,W 14,W 7,D 12,D 4,
1,2,DAKSHESH DARURI,6.0,W 63,W 58,L 4,W 17,W 16,W 20,W 7,
2,3,ADITYA BAJAJ,6.0,L 8,W 61,W 25,W 21,W 11,W 13,W 12,
3,4,PATRICK H SCHILLING,5.5,W 23,D 28,W 2,W 26,D 5,W 19,D 1,
4,5,HANSHI ZUO,5.5,W 45,W 37,D 12,D 13,D 4,W 14,W 17,


- split the result-letter-indicator from the ID of the opponent, using the `'extract()'` method with regex `'([0-9]+)'` to return only the ID.
- drop all the columns containing the data we don't need.
- give the columns containing the data we need meaningful names.

In [11]:
#we create a loop to extract only the ID of the opponents (the original data also has a result-letter-indicatorand),
#and save them in new columns.
#We used the index i both to name the columns (we added -2 because we wanted to have the names starting from 1),
#and to call the column.
for i in range(3,10):
        first_row[['ID_Opponent_R%s' %(i-2)]] =first_row.iloc[:,i].str.extract("([0-9]+)") 

#we drop all the columns not in our scope
first_row=first_row.drop(first_row.iloc[:,3:11],axis=1)

#rename columns 1,2 and 0
first_row.rename(columns={1:'Player_Name', 2:'Total_Points',0:'Player_ID'
                    },inplace = True)
#print dataframe to check result is as expected
first_row.head()

,Player_ID,Player_Name,Total_Points,ID_Opponent_R1,ID_Opponent_R2,ID_Opponent_R3,ID_Opponent_R4,ID_Opponent_R5,ID_Opponent_R6,ID_Opponent_R7
0,1,GARY HUA,6.0,39,21,18,14,7,12,4
1,2,DAKSHESH DARURI,6.0,63,58,4,17,16,20,7
2,3,ADITYA BAJAJ,6.0,8,61,25,21,11,13,12
3,4,PATRICK H SCHILLING,5.5,23,28,2,26,5,19,1
4,5,HANSHI ZUO,5.5,45,37,12,13,4,14,17


**The next steps are:**

- join the previously created two dataframes, to have all the necessary informations in the same row for each player.
- strip the columns of white space characters using a `lambda` statement to iterate the function `strip()`.
- join the dataframe with itself, to obtain new columns with the opponents' Pre_rating. 
- use the Pre_rating columns to calculate the average for each player.

In [12]:
#now, we join the 2 dataframes to have all the necessary information in the same row for each player.
#we use the index as the key for joining the 2 dataframes.
result = pd.concat([first_row, second_row], axis=1, join='inner')

#print dataframe to check result is as expected
result.head()

,Player_ID,Player_Name,Total_Points,ID_Opponent_R1,ID_Opponent_R2,ID_Opponent_R3,ID_Opponent_R4,ID_Opponent_R5,ID_Opponent_R6,ID_Opponent_R7,State,Pre_rating
0,1,GARY HUA,6.0,39,21,18,14,7,12,4,ON,1794
1,2,DAKSHESH DARURI,6.0,63,58,4,17,16,20,7,MI,1553
2,3,ADITYA BAJAJ,6.0,8,61,25,21,11,13,12,MI,1384
3,4,PATRICK H SCHILLING,5.5,23,28,2,26,5,19,1,MI,1716
4,5,HANSHI ZUO,5.5,45,37,12,13,4,14,17,MI,1655


- strip the columns of white space characters using a `lambda` statement to iterate the function `strip()`.

In [13]:
#now we need to take off all the white space before and after each string column
data_frame_trimmed = result.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

#print dataframe
data_frame_trimmed.head()

,Player_ID,Player_Name,Total_Points,ID_Opponent_R1,ID_Opponent_R2,ID_Opponent_R3,ID_Opponent_R4,ID_Opponent_R5,ID_Opponent_R6,ID_Opponent_R7,State,Pre_rating
0,1,GARY HUA,6.0,39,21,18,14,7,12,4,ON,1794
1,2,DAKSHESH DARURI,6.0,63,58,4,17,16,20,7,MI,1553
2,3,ADITYA BAJAJ,6.0,8,61,25,21,11,13,12,MI,1384
3,4,PATRICK H SCHILLING,5.5,23,28,2,26,5,19,1,MI,1716
4,5,HANSHI ZUO,5.5,45,37,12,13,4,14,17,MI,1655


- join the dataframe with itself, to obtain new columns with the opponents' Pre_rating. 

In [14]:
#we want to merge the dataframe with a subset of itself ('Player_ID','Pre_rating') for each round,
#to obtain a new column with the opponents Pre_rating.
#we need to use the ID_Opponent_R# as left key to join with the Player_ID, 
#and we want to extract the Pre_rating_#.
#to do that, we created a loop for the 7 opponents that we have and check when the opponent_ID matches with the Player_ID 
#in the subset. For each case we extract the corrisponding Pre_rating.
#Furthermore, considering the behaviour of the join when the columns have the same name, 
#which is rename the left as %s_x and the right as %s_y.
#we rename the columns using the index i to make them unique, and rename the Player_ID_x as Player_ID to enable the new join
#and each time we deleted the Player_ID_y which we don't need because it has the same value as ID_Opponent_R#.
for i in range(1,8):
    data_frame_trimmed=data_frame_trimmed.merge(data_frame_trimmed[['Player_ID','Pre_rating']],  how='left', left_on=['ID_Opponent_R%s' %i], right_on = ['Player_ID'])
    data_frame_trimmed.rename(columns={'Player_ID_x':'Player_ID','Pre_rating_x':'Pre_rating','Pre_rating_y':'Pre_rating_%s' %i 
                    },inplace = True)
    data_frame_trimmed=data_frame_trimmed.drop(columns=['Player_ID_y'],axis=1)

#print dataframe to check result is as expected
data_frame_trimmed.head()

,Player_ID,Player_Name,Total_Points,ID_Opponent_R1,ID_Opponent_R2,ID_Opponent_R3,ID_Opponent_R4,ID_Opponent_R5,ID_Opponent_R6,ID_Opponent_R7,State,Pre_rating,Pre_rating_1,Pre_rating_2,Pre_rating_3,Pre_rating_4,Pre_rating_5,Pre_rating_6,Pre_rating_7
0,1,GARY HUA,6.0,39,21,18,14,7,12,4,ON,1794,1436.0,1563.0,1600.0,1610.0,1649.0,1663.0,1716.0
1,2,DAKSHESH DARURI,6.0,63,58,4,17,16,20,7,MI,1553,1175.0,917.0,1716.0,1629.0,1604.0,1595.0,1649.0
2,3,ADITYA BAJAJ,6.0,8,61,25,21,11,13,12,MI,1384,1641.0,955.0,1745.0,1563.0,1712.0,1666.0,1663.0
3,4,PATRICK H SCHILLING,5.5,23,28,2,26,5,19,1,MI,1716,1363.0,1507.0,1553.0,1579.0,1655.0,1564.0,1794.0
4,5,HANSHI ZUO,5.5,45,37,12,13,4,14,17,MI,1655,1242.0,980.0,1663.0,1666.0,1716.0,1610.0,1629.0


- use the Pre_rating columns to calculate the average for each player.

In [15]:
#calculate the average Pre_rating of the opponents and add to column 'avg_opponent',
#for every player by iterating on every row in the dataframe,
#and applying the mean() method on the columns in the subset containing the Pre_ratings.
#also, set the average as an integer. 
for row in range (data_frame_trimmed.shape[0]):
    data_frame_trimmed['avg_opponent']=data_frame_trimmed.loc[:,'Pre_rating_1':'Pre_rating_7'].mean(axis=1,skipna = True).round().astype('int64')

#print dataframe to check result is as expected
data_frame_trimmed.head()

,Player_ID,Player_Name,Total_Points,ID_Opponent_R1,ID_Opponent_R2,ID_Opponent_R3,ID_Opponent_R4,ID_Opponent_R5,ID_Opponent_R6,ID_Opponent_R7,State,Pre_rating,Pre_rating_1,Pre_rating_2,Pre_rating_3,Pre_rating_4,Pre_rating_5,Pre_rating_6,Pre_rating_7,avg_opponent
0,1,GARY HUA,6.0,39,21,18,14,7,12,4,ON,1794,1436.0,1563.0,1600.0,1610.0,1649.0,1663.0,1716.0,1605
1,2,DAKSHESH DARURI,6.0,63,58,4,17,16,20,7,MI,1553,1175.0,917.0,1716.0,1629.0,1604.0,1595.0,1649.0,1469
2,3,ADITYA BAJAJ,6.0,8,61,25,21,11,13,12,MI,1384,1641.0,955.0,1745.0,1563.0,1712.0,1666.0,1663.0,1564
3,4,PATRICK H SCHILLING,5.5,23,28,2,26,5,19,1,MI,1716,1363.0,1507.0,1553.0,1579.0,1655.0,1564.0,1794.0,1574
4,5,HANSHI ZUO,5.5,45,37,12,13,4,14,17,MI,1655,1242.0,980.0,1663.0,1666.0,1716.0,1610.0,1629.0,1501


**The next and Final step is to create the .CSV file** 


In [16]:
#we create a copy of dataframe, to print only the columns in our scope.
df_toprint=data_frame_trimmed.drop(data_frame_trimmed.loc[:,'ID_Opponent_R1':'ID_Opponent_R7'],axis=1).copy()

#we drop all the Pre_rating_# columns
df_toprint=df_toprint.drop(df_toprint.loc[:,'Pre_rating_1':'Pre_rating_7'],axis=1)
df_toprint=df_toprint.drop(columns='Player_ID',axis=1)
df_toprint=df_toprint.iloc[:,[0,2,1,3,4]]
#rename the columns as requested
df_toprint.rename(columns={'Player_Name':'Player Name','Total_Points':'Total Number of Points','State':'Player State', 
                           'Pre_rating':'Player Pre-Rating','avg_opponent':'Average Pre Tournament Chess Rating of Opponents'
                   },inplace = True)

#print dataframe to check result is as expected
df_toprint.head()

,Player Name,Player State,Total Number of Points,Player Pre-Rating,Average Pre Tournament Chess Rating of Opponents
0,GARY HUA,ON,6.0,1794,1605
1,DAKSHESH DARURI,MI,6.0,1553,1469
2,ADITYA BAJAJ,MI,6.0,1384,1564
3,PATRICK H SCHILLING,MI,5.5,1716,1574
4,HANSHI ZUO,MI,5.5,1655,1501


In [17]:
#now that we checked that everything is as expected, we can extract the .csv file.
#we chose for the file to be extracted into the work folder of this Jupyter Notebook.
#this way, whoever runs this Jupyter notebook can have it at their disposal in their own folder.
df_toprint.to_csv('tournament.csv',index=False,sep=',')